# Master stats doc
Here I show the calculation of all the stats for my time series estimation work. These will be put into a function that returns all of them, which we will then calculate the error of for missing data. Note that streamlit code allows calculation for missing data

- Compare utils script to reynolds and update accordingly if needed


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import src.utils as utils
import glob
import pickle


c:\Users\spann\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Extract variables from CDF files and save as pickle files

In [2]:
def get_subfolders(path):
    return sorted(glob.glob(path + "/*"))


def get_cdf_paths(subfolder):
    return sorted(glob.iglob(subfolder + "/*.cdf"))


In [3]:
# mfi_file_list = get_cdf_paths("data/raw/wind/mfi/")
# proton_file_list = get_cdf_paths("data/raw/wind/3dp/")




In [4]:
# data_B = utils.pipeline(
#     mfi_file_list[0],
#     varlist=[params.timestamp, params_dict["Bwind, params_dict["Bwind_vec],
#     thresholds=params_dict["mag_thresh,
#     cadence=params_dict["dt_hr,
# )

# data_B = data_B.rename(columns={params_dict["Bx: "Bx", params_dict["By: "By", params_dict["Bz: "Bz"})
# data_B = data_B.interpolate(method="linear")
# mag_int_hr = data_B["2016-01-01 12:00":]

# print(mag_int_hr.head())




In [5]:
# mag_int_hr.to_pickle("data/processed/wind/mfi/2016-01-00_12:00.pkl")

In [6]:
# data_protons = utils.pipeline(
#     proton_file_list[0],
#     varlist=[
#         params.timestamp,
#         params.np,
#         params.nalpha,
#         params.Tp,
#         params.Talpha,
#         params.V_vec,
#     ],
#     thresholds=params.proton_thresh,
#     cadence=params.dt_protons,
# )

# data_protons = data_protons.rename(
#     columns={
#         params.Vx: "Vx",
#         params.Vy: "Vy",
#         params.Vz: "Vz",
#         params.np: "np",
#         params.nalpha: "nalpha",
#         params.Tp: "Tp",
#         params.Talpha: "Talpha",
#     }
# )

# data_protons = data_protons.interpolate(method="linear")
# proton_int = data_protons["2016-01-01 12:00":]

# print(proton_int.head())

# proton_int.to_pickle("data/processed/wind/3dp/2016-01-00_12:00.pkl")




LR is currently used for ACF and SF, HR for spectrum and taylor scale

In [7]:
def structure(data, ar_lags, ar_powers):
    """
    Routine to compute the Structure coefficients of a certain series or number of series to different orders
    of structure at lags given in ar_lags
    Input:
            data: pd.DataFrame of data to be analysed. Must have shape (1, N) or (3, N)
            ar_lags: The array consisting of lags, being the number of points to shift each of the series
            ar_powers: The array consisting of the Structure orders to perform on the series for all lags
    Output:
            df: The DataFrame containing the structure coefficients corresponding to ar_lags for each order in ar_powers
    """
    # run through ar_lags and ar_powers so that for each power, run through each lag
    df = {}

    if data.shape[1] == 1:
        ax = data.iloc[:, 0].copy()
        for i in ar_powers:
            array = []
            for l in ar_lags:
                dax = np.abs(ax.shift(-l) - ax)
                strct = dax.pow(i).mean()
                array += [strct]
            df[str(i)] = array

    elif data.shape[1] == 3:
        ax = data.iloc[:, 0].copy()
        ay = data.iloc[:, 1].copy()
        az = data.iloc[:, 2].copy()

        for i in ar_powers:
            array = []
            for l in ar_lags:
                dax = np.abs(ax.shift(-l) - ax)
                day = np.abs(ay.shift(-l) - ay)
                daz = np.abs(az.shift(-l) - az)
                strct = (dax.pow(2) + day.pow(2) + daz.pow(2)).pow(0.5).pow(i).mean()
                array += [strct]
            df[str(i)] = array

    df = pd.DataFrame(df)
    return df


In [13]:
def calculate_all_stats(time_series, var_name, params_dict):
    """
    Calculate a rangle of scale-domain statistics for a given time series
    :param time_series: list of pd.Series
    :param var_name: str
    :return: dict
    """
    # Compute autocorrelations and power spectra

    time_series_low_res = [x.resample(params_dict["dt_lr"]).mean() for x in time_series]

    # MATCHES UP WITH LINE 361 IN SRC/PROCESS_DATA.PY
    time_lags_lr, acf_lr = utils.compute_nd_acf(
        time_series=time_series_low_res, nlags=params_dict["nlags_lr"]
    )  # Removing "S" from end of dt string

    corr_scale_exp_trick = utils.compute_outer_scale_exp_trick(time_lags_lr, acf_lr)
    # corr_scale_exp_trick_list.append(corr_scale_exp_trick)

    # Use estimate from 1/e method to select fit amount
    corr_scale_exp_fit = utils.compute_outer_scale_exp_fit(
        time_lags_lr, acf_lr, np.round(2 * corr_scale_exp_trick)
    )
    # corr_scale_exp_fit_list.append(corr_scale_exp_fit)

    corr_scale_int = utils.compute_outer_scale_integral(time_lags_lr, acf_lr)
    # corr_scale_int_list.append(corr_scale_int)

    time_lags_hr, acf_hr = utils.compute_nd_acf(
        time_series=time_series, nlags=params_dict["nlags_hr"]
    )

    slope_k = np.nan
    # ~1min per interval due to spectrum smoothing algorithm
    try:
        (
            slope_i,
            slope_k,
            break_s,
            f_periodogram,
            power_periodogram,
            p_smooth,
            xi,
            xk,
            pi,
            pk,
        ) = utils.compute_spectral_stats(
            time_series=time_series,
            f_min_inertial=params_dict["f_min_inertial"],
            f_max_inertial=params_dict["f_max_inertial"],
            f_min_kinetic=params_dict["f_min_kinetic"],
            f_max_kinetic=params_dict["f_max_kinetic"],
        )

        # inertial_slope_list.append(slope_i)
        # kinetic_slope_list.append(slope_k)
        # spectral_break_list.append(break_s)

    except Exception as e:
        print("Error: spectral stats calculation failed: {}".format(e))
        # print("Interval timestamp: {}".format(int_start))
        # inertial_slope_list.append(np.nan)
        # kinetic_slope_list.append(np.nan)
        # spectral_break_list.append(np.nan)
        # slope_k = None

    if params_dict["tau_min"] is not None:
        taylor_scale_u, taylor_scale_u_std = utils.compute_taylor_chuychai(
            time_lags_hr,
            acf_hr,
            tau_min=params_dict["tau_min"],
            tau_max=params_dict["tau_max"],
        )

        if not np.isnan(slope_k):
            taylor_scale_c, taylor_scale_c_std = utils.compute_taylor_chuychai(
                time_lags_hr,
                acf_hr,
                tau_min=params_dict["tau_min"],
                tau_max=params_dict["tau_max"],
                q=slope_k,
            )

        else:
            taylor_scale_c = np.nan
            taylor_scale_c_std = np.nan
    else:
        taylor_scale_u = np.nan
        taylor_scale_u_std = np.nan
        taylor_scale_c = np.nan
        taylor_scale_c_std = np.nan

    int_lr_df = pd.concat(time_series_low_res, axis=1)
    sfns = structure(int_lr_df, np.arange(1, round(0.2 * len(int_lr_df))), [1, 2, 3, 4])

    # Calculate kurtosis (currently not component-wise)
    sdk = sfns[["2", "4"]]
    sdk.columns = ["2", "4"]
    sdk["2^2"] = sdk["2"] ** 2
    kurtosis = sdk["4"].div(sdk["2^2"])

    # Store these results in a dictionary
    stats_dict = {
        var_name: {
            "time_series": time_series_low_res,
            "times": time_lags_lr,
            "time_lags_hr": time_lags_hr,
            "xi": xi,
            "xk": xk,
            "pi": pi,
            "pk": pk,
            "cr": acf_lr,
            "acf_hr": acf_hr,
            "qi": slope_i,
            "qk": slope_k,
            "break_s": break_s,
            "corr_scale_exp_trick": corr_scale_exp_trick,
            "corr_scale_exp_fit": corr_scale_exp_fit,
            "corr_scale_int": corr_scale_int,
            "taylor_scale_u": taylor_scale_u,
            "taylor_scale_u_std": taylor_scale_u_std,
            "taylor_scale_c": taylor_scale_c,
            "taylor_scale_c_std": taylor_scale_c_std,
            "f_periodogram": f_periodogram,
            "power_periodogram": power_periodogram,
            "p_smooth": p_smooth,
            "sfn": sfns,  # multiple orders
            "sdk": kurtosis,
        }
    }

    return int_lr_df, stats_dict


In [9]:
def plot_stats(stats_dict):
    # Plot the results
    fig, axs = plt.subplots(2, 3, figsize=(15, 6))
    # Adjust the vertical spacing
    fig.subplots_adjust(hspace=0.4, wspace=0.4)

    # Time Series in the first panel
    axs[0, 0].plot(
        np.arange(len(stats_dict["time_series"])),
        stats_dict["time_series"],
        color="blue",
    )
    axs[0, 0].set_title("Time series $x(t)$")

    # ACF LR in the second panel
    axs[0, 1].plot(stats_dict["times"], stats_dict["cr"], color="green")
    axs[0, 1].axvline(stats_dict["corr_scale_exp_trick"], color="black", ls="--")
    axs[0, 1].axhline(np.exp(-1), color="black", ls="--")
    axs[0, 1].set_title("Autocorrelation $R(\\tau)$ (low-res)")

    # ACF HR in the third panel
    axs[0, 2].plot(stats_dict["time_lags_hr"], stats_dict["acf_hr"], color="red")
    axs[0, 2].set_title("Autocorrelation $R(\\tau)$ (high-res)")

    # Periodogram in the fourth panel
    axs[1, 0].plot(
        stats_dict["f_periodogram"],
        stats_dict["power_periodogram"],
        c="purple",
        alpha=0.5,
    )
    axs[1, 0].plot(stats_dict["f_periodogram"], stats_dict["p_smooth"], c="purple")
    axs[1, 0].plot(stats_dict["xi"], stats_dict["pi"] * 3, c="black", ls="--")
    axs[1, 0].plot(stats_dict["xk"], stats_dict["pk"] * 3, c="black", ls="--")
    axs[1, 0].axvline(stats_dict["break_s"], color="black", ls="dotted")
    axs[1, 0].set_title("Periodogram $P(f)$")
    axs[1, 0].set_xscale("log")
    axs[1, 0].set_yscale("log")

    # SFN in the fifth panel
    axs[1, 1].plot(stats_dict["sfn"], color="orange")
    axs[1, 1].set_title("Structure function $S_2(\\tau)$")
    axs[1, 1].set_xscale("log")
    axs[1, 1].set_yscale("log")

    # Kurtosis in the sixth panel
    axs[1, 2].plot(stats_dict["sdk"], color="cyan")
    axs[1, 2].set_title("Kurtosis $K(\\tau)=S_4/S_2^2$")

    plt.show()

In [14]:
mag_int_hr = pd.read_pickle("data/processed/wind/mfi/2016-01-00_12:00.pkl")

# Frequency bounds are taken from Wang et al. (2018, JGR)
mag_params = {
    "f_min_inertial": 0.005,
    "f_max_inertial": 0.2,
    "f_min_kinetic": 0.5,
    "f_max_kinetic": 1.4,
    "nlags_lr": 2000,
    "nlags_hr": 100,
    "dt_lr": "5S",
    "tau_min": 10,
    "tau_max": 50,
}

proton_int = pd.read_pickle("data/processed/wind/3dp/2016-01-00_12:00.pkl")
proton_params = {
    "f_min_inertial": None,
    "f_max_inertial": None,
    "f_min_kinetic": None,
    "f_max_kinetic": None,
    "nlags_lr": 2000,
    "nlags_hr": 100,
    "dt_lr": "5S",
    "tau_min": None,
    "tau_max": None,
}


In [15]:
# Get raw time series and turbulent quantities
flr, flt = calculate_all_stats([mag_int_hr.Bx], "Bx", mag_params)


C:\Users\spann\AppData\Local\Temp\ipykernel_12956\946917652.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdk["2^2"] = sdk["2"] ** 2


In [23]:
# Save dictionary for later plotting
flr.to_pickle("data/processed/" + "Bx_raw.pkl")
with open("data/processed/" + "Bx_turb.pkl", "wb") as file:
    pickle.dump(flt, file)


In [24]:
# Get raw time series and turbulent quantities
flr, flt = calculate_all_stats(
    [mag_int_hr.Bx, mag_int_hr.By, mag_int_hr.Bz], "B", mag_params
)


C:\Users\spann\AppData\Local\Temp\ipykernel_12956\946917652.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdk["2^2"] = sdk["2"] ** 2


In [25]:
flr.to_pickle("data/processed/" + "B_raw.pkl")
with open("data/processed/" + "B_turb.pkl", "wb") as file:
    pickle.dump(flt, file)


In [26]:
# Get raw time series and turbulent quantities
flr, flt = calculate_all_stats(
    [proton_int.Vx, proton_int.Vy, proton_int.Vz], "V", proton_params
)


C:\Users\spann\AppData\Local\Temp\ipykernel_12956\946917652.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdk["2^2"] = sdk["2"] ** 2


In [27]:
import pickle as pickle

flr.to_pickle("data/processed/" + "proton_raw.pkl")
with open("data/processed/" + "proton_turb.pkl", "wb") as file:
    pickle.dump(flt, file)
